# Laser Machine Listener

The following code implements machine learning with Keras (and TensorFlow as a backend) to classify sounds emitted by a laser machine during processing such as cutting and marking. The assumed states are as follows. As learning data, prepare folders as many as the number of states in the **data** folder, and place one or more sound files corresponding to each state in each folder.

- Background
- Cutting (in focus)
- Cutting (not in focus)
- Marking
- Waiting
- Sleeping

以下のコードは、レーザー加工機がカットやマーキングなどの加工中に発する音を分類するための機械学習をKeras（とバックエンドとしてのTensorFlow）で実装したものです。想定している状態は以下の通りです。学習用のデータとして、**data**フォルダの中に状態の数だけフォルダを用意し、それぞれのフォルダの中にそれぞれの状態に対応する1つまたはそれ以上の音声ファイルを配置してください。

- 背景音
- カット中（焦点が合っている）
- カット中（焦点が合っていない）
- マーキング中
- 待機中
- スリープ中

| State | Laser Machine | Dust collector | Ventilator |
| --: | :-- | :-- | :-- |
| Background | OFF | OFF | OFF |
| Sleeping | OFF | OFF | ON |
| Waiting | ON | ON | ON |
| Cutting (in focus) | ON | ON | ON |
| Cutting (not in focus) | ON | ON | ON |
| Marking | ON | ON | ON |

First, we initialise an array for containing state names and a matrix for storing voice data and correct labels. Next, read sound files matching the pattern of the path for 30 seconds in order, normalize, obtain STFT, and register together with the same number of correct answer labels.

まず、状態名を収めるための配列と、音声データと正解ラベルを収めるための行列を初期化する。次に、パスのパターンに合致する音声ファイルを順に30秒間ずつ読み込み、正規化したのちにSTFTを求め、同数の正解ラベルと共に登録する。

In [ ]:
import numpy as np
import glob
import librosa

SAMPLING_RATE = 16000
FFT_SIZE = 256
STFT_MATRIX_SIZE = 1 + FFT_SIZE // 2

state_names = []
data = np.empty((0, STFT_MATRIX_SIZE), dtype=np.float32)
index = np.empty(0, dtype=np.int32)

for path_name in sorted(glob.glob('data/*/*.wav')):
    state_name = path_name.split('/')[1]

    if state_name not in state_names:
        state_names.append(state_name)

    audio, sr = librosa.load(path_name, sr=SAMPLING_RATE, duration=30)
    print('{}: {} ({} Hz) '.format(state_name, path_name, sr))
    d = np.abs(librosa.stft(librosa.util.normalize(audio),
                            n_fft=FFT_SIZE, window='hamming'))
    data = np.vstack((data, d.transpose()))
    index = np.hstack([index, [state_names.index(state_name)] * d.shape[1]])

To check what kind of difference is seen when viewing each state on the frequency axis, we calculate the average of STFT for each state and draw with matplotlib.

それぞれの状態を周波数軸で見たときにどのような違いがあるのかを確認するため、状態ごとにSTFTの平均を求め、matplotlibを用いて描画します。

In [ ]:
import matplotlib.pyplot as plt

n_states = len(state_names)

fig = plt.figure(figsize=(15, 5 * n_states))

for i, state_name in enumerate(state_names):
    plt.subplot(n_states, 1, 1 + i)
    plt.plot(librosa.fft_frequencies(sr=SAMPLING_RATE, n_fft=FFT_SIZE),
             np.mean(data[np.where(index == i)], axis=0))
    plt.title(state_name)

plt.show()

Now the data was prepared as above. First, define a model of the neural network with Keras, and divide the data and the correct index into 80% for learning and the remaining 20% for verification. Next, Select Adam as an optimisation method, set the number of epochs to 20, and set an early stopping when there is no change in the correct answer rate, and execute learning. When using the files bundled as samples, the correct answer rate is about 95%. If you do not get the proper answer rate for the examples you prepared, try tuning parameters such as `N_MID_UNITS` and `BATCH_SIZE`.

以上でデータが用意できました。まず、Kerasでニューラルネットワークのモデルを定義し、データと正解インデックスをのうち、8割を学習用、残りの2割を検証用に分けます。次に、最適化手法としてAdamを選択、エポック数は20に設定し、正解率に変化がなくなったら早期に終了するように設定した上で学習を実行します。サンプルとして同梱しているファイルを用いた場合、正解率は約95%程度になります。もし、自分で用意したサンプルに対する正解率が上がらない場合には、`N_MID_UNITS`や`BATCH_SIZE`などのパラメータをチューニングしてみましょう。

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

N_MID_UNITS = 128

model = Sequential()
model.add(Dense(N_MID_UNITS, activation='sigmoid', input_dim=STFT_MATRIX_SIZE))
model.add(Dense(N_MID_UNITS // 2, activation='sigmoid'))
model.add(Dense(n_states, activation='softmax'))
model.compile(Adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


from sklearn.model_selection import train_test_split

index_cat = keras.utils.to_categorical(index)
data_train, data_test, index_train, index_test = train_test_split(data, index_cat, test_size=0.2)

from keras.callbacks import EarlyStopping

es_callback = EarlyStopping(monitor='val_acc',
                            patience=2,
                            verbose=True,
                            mode='auto')

BATCH_SIZE = 32

model.fit(data_train, index_train, epochs=20, batch_size=BATCH_SIZE,
          validation_split=0.2, callbacks=[es_callback])

When learning is over, test with the data and correct answer index that have been separated for verification.

学習が終わったら、検証のために分離しておいたデータと正解インデックスを用いて確認します。

In [ ]:
model.evaluate(data_test, index_test)

Finally, save the array containing the state name and the learned model as separate files.

最後に、状態の名前を収めた配列と学習済みのモデルを個別のファイルとして保存します。

In [ ]:
import json

with open('state_names.json', 'w') as file_to_save:
    json.dump(state_names, file_to_save)

model.save('laser_machine_listener.h5')